In [1]:
%load_ext autoreload
%autoreload 2
import os
import time
import jax

import numpy as np
import pandas as pd
import sys
import yaml
jax.config.update("jax_enable_x64", True)
import pickle

In [ ]:
# Install dependencies
!pip install linearmodels

In [ ]:
!conda install estimagic -c conda-forge

In [ ]:
import estimagic as em

In [2]:
# Mount drive. Make sure everything(including submodules) are there
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
%cd drive/MyDrive/colab_notebooks/policy_unc/analysis

/content/drive/MyDrive/colab_notebooks/policy_unc/analysis


In [ ]:
analysis_path = os.getcwd() + "/"
file_dir_path = analysis_path + "src/estimation/"
project_paths = {
    "project_path": analysis_path,
    "model_path": file_dir_path + "results_and_data/",
}
from estimation.tools import process_data_and_model


In [ ]:
sys.path.insert(0, analysis_path + "submodules/dcegm/src/")
sys.path.insert(0, analysis_path + "src/")
data_decision = pd.read_pickle(analysis_path + "output/decision_data.pkl")

In [ ]:
start_params_all = {
    # Utility parameters
    "mu": 0.5,
    "dis_util_work": 4.0,
    "dis_util_unemployed": 1.0,
    "bequest_scale": 2.0,
    # Taste and income shock scale
    "lambda": 1.0,
    # Interest rate and discount factor
    "interest_rate": 0.03,
    "beta": 0.95,
}


params_to_estimate_names = [
    # "mu",
    "dis_util_work",
    "dis_util_unemployed",
    "bequest_scale",
    # "lambda",
    # "sigma",
]
start_params = {name: start_params_all[name] for name in params_to_estimate_names}


lower_bounds = {
    "dis_util_work": 1e-12,
    "dis_util_unemployed": 1e-12,
    "bequest_scale": 1e-12,
    # "lambda": 1e-12,
}
upper_bounds = {
    "dis_util_work": 100,
    "dis_util_unemployed": 100,
    "bequest_scale": 10,
    # "lambda": 100,
}

individual_likelihood = process_data_and_model(
    data_decision=data_decision,
    project_paths=project_paths,
    start_params_all=start_params_all,
    load_model=True,
)


def individual_likelihood_print(params):
    ll_value = individual_likelihood(params)
    print("Params, ", params, " with ll value, ", ll_value)
    return ll_value




In [ ]:
result = em.minimize(
    criterion=individual_likelihood_print,
    params=start_params,
    lower_bounds=lower_bounds,
    upper_bounds=upper_bounds,
    algorithm="scipy_lbfgsb",
    logging="test_log.db",
    error_handling="continue",
)
pickle.dump(result, open(file_dir_path + "res.pkl", "wb"))